In [37]:
import jellyfish
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import dateutil.parser as parser
df = pd.read_excel("NUS sample names_V2.xlsx", engine="openpyxl")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lionel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Web Scraper

In [4]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lionel/nltk_data...


In [166]:
# Main Function
def search_articles_on_individual(individual_dict, no_of_articles=10):
    def generate_link(person_dict, attributes_used = ['name', 'alias'], keywords=['crimes', 'sentenced']):
        link_start = "https://www.google.com/search?q="
        link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
        link_query = ""

        for attributes in attributes_used:
            temp_attr = person_dict[attributes]
            if temp_attr is not None:
                temp_attr = str(temp_attr)
                link_query += temp_attr.replace(' ', '+') + '+'       
                
        links = []
        for keyword in keywords:
            temp_search_link = link_start + link_query + keyword + link_end + "&num=" + str(no_of_articles)
            links.append(temp_search_link)
        return links
    
    def article_extraction(link):
        article = Article(link)
        article.download()
        try:
            article.parse()
        except:
            pass
        return article.text
    
    def sentiment_analysis(text):
        return vader.polarity_scores(text)

    
    link = generate_link(individual_dict)
    
    unique_links_checker = []
    
    output = []
    for x in link:
        print("SEARCHING THIS", x)
        req = Request(x, headers = {'User-Agent': 'Mozilla/5.0'})

        webpage = urlopen(req).read()

        links = []
        with requests.Session() as c:
            soup = BeautifulSoup(webpage, 'html5lib')
            #print(soup)
            for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
                current_dict = {}
                raw_link = (item.find('a', href = True)['href'])
                link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
                if link not in unique_links_checker:
                    unique_links_checker.append(link)
                    title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).get_text()
                    description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
                    time = description.split(" · ")[0]
                    #print(description)
                    descript = description.split(" · ")[1]
                    current_dict['title'] = title
                    current_dict['time'] = time
                    current_dict['description'] = descript
                    current_dict['link'] = link
                    current_dict['text'] = article_extraction(link)
                    # current_dict['sentiment?'] = sentiment_analysis(article_extraction(link))
                    output.append(current_dict)
                else:
                    pass
    return output


new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'bankrupt': -100,
    'fraud': -100,
    'crime': -100,
    'prison': -100,
    'embezzlement': -100,
    'money_trail':-100,
    'jail': -100,
    'sentenced':-100,
    'forfeiture': -100,
    'ill-gotten gain':-100,
    'laundering':-100
}


vader = SentimentIntensityAnalyzer()

vader.lexicon.update(new_words)

In [164]:
def preprocess_df_to_dict(df):
    def get_year(date):
        try:
            parser_obj = parser.parse(str(date))
            return parser_obj.year
        except:
            return None
    def get_month(date):
        if len(str(date))>4:
            try:
                return parser.parse(str(date)).month
            except:
                return None
        else:
            return None
    def get_day(date):
        if len(str(date))>4:
            try:
                return parser.parse(str(date)).day
            except:
                return None
        else:
            return None    
    
    df_dict_list = df.to_dict('records')
    cleaned_dict_list = []
    for record in df_dict_list:   
        current_record = {
            'name': record['Name to be screened'],
            'alias' : None if 'nan' else record['alias'],
            'year_of_birth': get_year(record['Date of birth']),
            'month_of_birth': get_month(record['Date of birth']),
            'day_of_birth': get_day(record['Date of birth']),
            'gender': None if 'nan' else record['Gender'],
            'nationality': None if 'nan' else record['Nationality'],
            ### delete these later on, for testing only###
            'type_of_error': record['Type of variation (if any)'],
            'actual_name': record['Actual name'],
        }
        cleaned_dict_list.append(current_record)
    return cleaned_dict_list

In [165]:
df_dict = preprocess_df_to_dict(df)
test_record_1 = df_dict[0]
print(test_record_1)

{'name': 'Chia Teck Leng', 'alias': None, 'year_of_birth': 1960, 'month_of_birth': None, 'day_of_birth': None, 'gender': None, 'nationality': None, 'type_of_error': '-', 'actual_name': 'Chia Teck Leng'}


In [168]:
test_query = search_articles_on_individual(test_record_1, 10)
pd.DataFrame(test_query)

SEARCHING THIS https://www.google.com/search?q=Chia+Teck+Leng+crimes&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2&num=10
SEARCHING THIS https://www.google.com/search?q=Chia+Teck+Leng+sentenced&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2&num=10


,title,time,description,link,text,sentiment?
0,Guilty As Charged: Chia Teck Leng led a double...,63 months ago,Asia Pacific Breweries executive Chia Teck Len...,https://www.straitstimes.com/singapore/courts-...,This story was first published in July 2015 in...,"{'neg': 0.453, 'neu': 0.495, 'pos': 0.052, 'co..."
1,Guilty As Charged: Shocking crimes that have s...,63 months ago,Chia Teck Leng secretly led a double life as a...,https://www.straitstimes.com/singapore/courts-...,The following stories were first published in ...,"{'neg': 0.47, 'neu': 0.498, 'pos': 0.031, 'com..."
2,John Soh could face longest-ever jail term for...,54 months ago,"SINGAPORE (March 3): Chia Teck Leng, a former ...",https://www.theedgemarkets.com/article/john-so...,"SINGAPORE (March 3): Chia Teck Leng, a former ...","{'neg': 0.519, 'neu': 0.464, 'pos': 0.017, 'co..."
3,Accountant who embezzled $40m a compulsive gam...,27 months ago,While Ewe did not amass riches from his crimes...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - Certified public accountant Ewe Pa...,"{'neg': 0.322, 'neu': 0.648, 'pos': 0.03, 'com..."
4,"Sherlock Holmes of Shenton Way, Brunch - THE B...",38 months ago,PwC forensics leader Chan Kheng Tek adds that ...,https://www.businesstimes.com.sg/brunch/sherlo...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
5,"National Day Awards 2021 (3), Singapore News &...",1 month ago,... Yio Chu Kang Secondary School; Mdm Leong P...,https://www.straitstimes.com/singapore/nationa...,"Mdm Emilia Bte Razali, Level Head, Si Ling Pri...","{'neg': 0.148, 'neu': 0.826, 'pos': 0.026, 'co..."
6,A higher standard of financial prudence and re...,122 months ago,Chia Teck Leng (“Chia”) was the finance manage...,https://www.lexology.com/library/detail.aspx%3...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
7,Accountant who embezzled $40 million a compuls...,27 months ago,"In March, he was convicted of 50 charges of cr...",https://www.asiaone.com/singapore/accountant-w...,SINGAPORE - Certified public accountant Ewe Pa...,"{'neg': 0.326, 'neu': 0.645, 'pos': 0.029, 'co..."
8,Traffic police officer dies in accident in Ser...,51 months ago,— with Chia So Chai and Leng Lim. 4.6K. 1.7K,https://www.straitstimes.com/singapore/acciden...,SINGAPORE - A traffic police officer died in a...,"{'neg': 0.182, 'neu': 0.779, 'pos': 0.039, 'co..."
9,"When being mobile means quality of life, Lates...",40 months ago,"Mr Jason Peh, an occupational therapist at Kho...",https://www.tnp.sg/news/singapore/when-being-m...,“It allows them to connect with their social n...,"{'neg': 0.026, 'neu': 0.894, 'pos': 0.08, 'com..."
